In [12]:
import snowflake.connector
import os
import pandas as pd
from snowflake.connector.errors import NotSupportedError

def sf_query(sql: str):
    """
    Ejecuta UNA sola sentencia SQL en Snowflake.
    - Si trae resultados, devuelve un DataFrame.
    - Si no, imprime confirmación.
    Maneja SHOW/DESCRIBE/SELECT aunque fetch_pandas_all no esté soportado.
    """
    conn = snowflake.connector.connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        account=os.getenv("SNOWFLAKE_HOST").split(".snowflakecomputing.com")[0],
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database=os.getenv("SNOWFLAKE_DATABASE"),
        role=os.getenv("SNOWFLAKE_ROLE"),
    )
    try:
        cur = conn.cursor()
        # Intentamos usar Arrow para que fetch_pandas_all funcione
        try:
            cur.execute("ALTER SESSION SET QUERY_RESULT_FORMAT=ARROW")
        except Exception:
            pass  # si falla, igual seguimos (haremos fallback)

        cur.execute(sql)

        if cur.description:  # hay result set (SELECT/SHOW/DESCRIBE)
            try:
                df = cur.fetch_pandas_all()  # rápido si hay Arrow/pyarrow
            except NotSupportedError:
                # Fallback sin Arrow
                rows = cur.fetchall()
                cols = [d[0] for d in cur.description]
                df = pd.DataFrame(rows, columns=cols)
            finally:
                cur.close()
            return df
        else:
            cur.close()
            print("Executed successfully.")
    finally:
        conn.close()


In [18]:
sf_query("""
CREATE OR REPLACE VIEW ANALYTICS.OBT_TRIPS_V AS
SELECT
  service_type,
  pickup_datetime,
  dropoff_datetime,
  DATEDIFF('minute', pickup_datetime, dropoff_datetime)                         AS trip_minutes,
  EXTRACT(HOUR FROM pickup_datetime)                                            AS pickup_hour,
  TO_CHAR(pickup_datetime, 'DY')                                                AS pickup_dow,
  IFF(DAYOFWEEK(pickup_datetime) IN (0, 6), TRUE, FALSE)                        AS is_weekend,
  trip_distance,
  pu_location_id,
  do_location_id,
  fare_amount,
  tip_amount,
  tolls_amount,
  mta_tax,
  improvement_surcharge,
  congestion_surcharge,
  total_amount,
  payment_type,
  vendor_id,
  rate_code_id,
  store_and_fwd_flag,
  year,
  month,
  IFF(NULLIFZERO(fare_amount) IS NULL, NULL, tip_amount / NULLIFZERO(fare_amount)) AS tip_pct
FROM RAW.TRIPS_ALL;
""")


,status
0,View OBT_TRIPS_V successfully created.


In [15]:
sf_query("SHOW VIEWS IN SCHEMA ANALYTICS;")


,created_on,name,reserved,database_name,schema_name,owner,comment,text,is_secure,is_materialized,owner_role_type,change_tracking
0,2025-10-23 08:38:03.582000-07:00,OBT_TRIPS_V,,DM_PSET3,ANALYTICS,SYSADMIN,,CREATE OR REPLACE VIEW ANALYTICS.OBT_TRIPS_V A...,false,false,ROLE,OFF


In [16]:
sf_query("SELECT COUNT(*) AS cnt FROM ANALYTICS.OBT_TRIPS_V;")


,CNT
0,851622729


In [17]:
sf_query("SELECT COUNT(*) AS cnt FROM RAW.TRIPS_ALL;")


,CNT
0,851622729


In [21]:
sf_query("SELECT * FROM ANALYTICS.OBT_TRIPS_V LIMIT 20;")


,SERVICE_TYPE,PICKUP_DATETIME,DROPOFF_DATETIME,TRIP_MINUTES,PICKUP_HOUR,PICKUP_DOW,IS_WEEKEND,TRIP_DISTANCE,PU_LOCATION_ID,DO_LOCATION_ID,...,IMPROVEMENT_SURCHARGE,CONGESTION_SURCHARGE,TOTAL_AMOUNT,PAYMENT_TYPE,VENDOR_ID,RATE_CODE_ID,STORE_AND_FWD_FLAG,YEAR,MONTH,TIP_PCT
0,yellow,2015-01-01 03:05:17,2015-01-01 03:33:46,28,3,Thu,False,7.10,166,7,...,0.0,NaN,28.93,1,1,1,N,2015,1,0.105200
1,yellow,2015-01-01 03:33:16,2015-01-01 03:35:16,2,3,Thu,False,0.50,107,107,...,0.0,NaN,6.80,1,1,1,N,2015,1,0.571429
2,yellow,2015-01-01 03:56:14,2015-01-01 04:04:54,8,3,Thu,False,4.26,7,70,...,0.3,NaN,14.80,2,2,1,N,2015,1,0.000000
3,yellow,2015-01-01 03:06:20,2015-01-01 03:13:19,7,3,Thu,False,1.27,7,179,...,0.3,NaN,7.80,2,2,1,N,2015,1,0.000000
4,yellow,2015-01-01 03:12:36,2015-01-01 03:22:41,10,3,Thu,False,3.90,161,41,...,0.0,NaN,16.55,1,1,1,N,2015,1,0.220000
5,yellow,2015-01-01 03:03:05,2015-01-01 03:18:23,15,3,Thu,False,4.06,263,79,...,0.3,NaN,18.20,1,2,1,N,2015,1,0.207143
6,yellow,2015-01-01 03:01:54,2015-01-01 03:07:06,6,3,Thu,False,0.95,229,163,...,0.3,NaN,6.80,2,2,1,N,2015,1,0.000000
7,yellow,2015-01-01 03:09:21,2015-01-01 03:26:29,17,3,Thu,False,5.20,264,264,...,0.0,NaN,18.50,2,1,1,N,2015,1,0.000000
8,yellow,2015-01-01 03:32:01,2015-01-01 03:36:17,4,3,Thu,False,0.64,41,41,...,0.3,NaN,6.30,2,2,1,N,2015,1,0.000000
9,yellow,2015-01-01 03:08:35,2015-01-01 03:20:12,12,3,Thu,False,2.00,238,48,...,0.0,NaN,14.65,1,1,1,N,2015,1,0.335000


In [22]:
sf_query("""
SELECT
  SUM(IFF(trip_minutes < 0, 1, 0)) AS trips_negativos,
  SUM(IFF(tip_pct < 0, 1, 0))      AS tips_negativos,
  SUM(IFF(tip_pct > 5, 1, 0))      AS tips_mayor_500pct
FROM ANALYTICS.OBT_TRIPS_V;
""")


,TRIPS_NEGATIVOS,TIPS_NEGATIVOS,TIPS_MAYOR_500PCT
0,13035,28116,123109
